In [1]:
# Add relative path to ExoJulia
push!(LOAD_PATH, "../../../ExoJulia/")

# import
using ExoJulia

In [2]:
using LsqFit
using PyPlot

In [34]:
# From time to M
M(t::Array,t_p::Float64,P::Float64) = (2*pi/P).*(t.-t_p)
M(t::Float64,t_p::Float64,P::Float64) = (2*pi/P)*(t-t_p)

M (generic function with 2 methods)

In [31]:
#How to convert from E to f
EtoF(E::Float64,ecc::Float64) = 2.*atan((((1.+ecc)/(1.-ecc))^(1./2.))*tan(E/2.))
EtoF(E::Array,ecc::Float64) = 2.*atan((((1.+ecc)/(1.-ecc))^(1./2.)).*tan(E./2.))

EtoF (generic function with 2 methods)

In [5]:
K(h::Float64,c::Float64) = sqrt(h*h+c*c)

K (generic function with 1 method)

In [6]:
pomega(h::Float64,c::Float64) = atand(-c/h)

pomega (generic function with 1 method)

In [7]:
gamma(v0::Float64,K::Float64,ecc::Float64,pomega::Float64) = v0-K*ecc*cosd(pomega)

gamma (generic function with 1 method)

In [8]:
function P_guesser(x_data::Array,y_data::Array)
    
    #minimum period to look for is distance between data points. Let's put the distances in an array
    x_dist=[]
    for i=2:size(x_data)[1]
        push!(x_dist,x_data[i]-x_data[i-1])
    end
    
    sq_array = []
    P_array = []
    
    for P=mean(x_dist):0.5:maximum(x_data)-minimum(y_data)
        
        phases = mod(x_data,P)
        RV_sort = y_data[sortperm(phases)] 
        
        sq = 0.0
        for i=2:size(RV_sort)[1]
            sq += (RV_sort[i]-RV_sort[i-1])^2.0
        end
        
        push!(sq_array,sq)
        push!(P_array,P)
    end
    
    return P_array[findmin(sq_array)[2]]
    
end

P_guesser (generic function with 1 method)

In [9]:
data = readdlm("mystery_planet.txt",Float64) ;

In [10]:
times = data[:,1]
RVs = data[:,2]
RV_errs = data[:,3] ;

In [33]:
function kepler_solve!(Ms::Array{Float64},ecc::Float64)
    results = Float64[]
    for em in Ms
        push!(results,ExoJulia.Orbit.kepler_solve!(em, ecc))
    end
    return results
end

kepler_solve! (generic function with 1 method)

In [62]:
function fit_RV(x_data,y_data,y_err)
    
    P_guess = P_guesser(x_data,y_data)
    
    #Because h, c and v0 depend on the data, we'll need to define the function when we fit
    
    function hcv0(f::Array)
        W = diagm(1.0./(y_err.^2.0))
        F = hcat(cos(f),sin(f),[1.0 for x in f])'
        epsilon = inv(F*W*F')
        return y_data'*W*F'*epsilon
    end
    
    function v_rad(t::Array,params) #params are e,t_p,P
        if 0 <= params[1] < 1
            Es = kepler_solve!(M(t,params[2],params[3]),params[1])
            f = EtoF(Es,params[1])
            h,c,v0 = hcv0(f)
            return h.*cos(f)+c*sin(f).+v0
        else
            return Inf
        end
    end
    
    function get_params(fit_obj::LsqFit.LsqFitResult{Float64}) #returns e,t_p,P,K,pomega,gamma
        params = fit_obj.param
        ecc = params[1]
        teepee = params[2]
        per = params[3]
        Es = kepler_solve!(M(x_data,teepee,per),ecc)
        f = EtoF(Es,ecc)
        h,c,v0 = hcv0(f)
        kay = K(h,c)
        pomeg = pomega(h,c)
        gam = gamma(v0,kay,ecc,pomeg)
        return ecc,teepee,per,kay,pomeg,gam
    end
        
    p0 = [0.1,x_data[1],P_guess]
    return get_params(curve_fit(v_rad,x_data,y_data,(1.0./y_err.^2.0),p0))
    
end

fit_RV (generic function with 1 method)

In [63]:
result = fit_RV(times,RVs,RV_errs) ;

In [64]:
param_names = ["e","t_p","P","K","pomega","gamma"]
for (param,res) in zip(param_names,result)
    println("$param = $res")
end

e = 0.40016612320726774
t_p = 13891.356528538307
P = 116.68569506349166
K = 316.1090092004665
pomega = -0.7766860911621005
gamma = -51.916669565780765


In [66]:
function eff_func(t,t0::Float64,r0::Float64,sma::Float64,ecc::Float64,t_p::Float64,P::Float64)
    Ms = M(t,t_p,P)
    M0 = M(t0,t_p,P)
    E0 = kepler_solve!(M0,ecc)
    E = kepler_solve!(Ms,ecc)
    return (sma/r0).*(cos(E.-E0).+1)
end

eff_func (generic function with 1 method)

In [69]:
function gee_func(t,t0::Float64,r0::Float64,sma::Float64,ecc::Float64,t_p::Float64,P::Float64)
    Ms = M(t,t_p,P)
    M0 = M(t0,t_p,P)
    E0 = kepler_solve!(M0,ecc)
    E = kepler_solve!(Ms,ecc)
    return (t.-t0).+(P/(2.0*pi)).*(sin(E.-E0)-(E.-E0))
end

gee_func (generic function with 1 method)